In [1]:
import pandas as pd

In [4]:
df_games = pd.read_parquet('../Datasets/steam_games_preprocessed.parquet')
df_user_items = pd.read_parquet('../Datasets/users_items_preprocessed.parquet')

In [5]:
df_games = df_games.rename(columns={'title': 'item_name'})
df_merged = pd.merge(df_user_items, df_games[['item_name', 'release_date', 'genres']], on='item_name')

In [6]:
df_merged.head()

,item_name,playtime_forever,user_id,items_count,release_date,genres
0,Counter-Strike,6,76561197970982479,277,2000-11-01,Action
1,Team Fortress Classic,0,76561197970982479,277,1999-04-01,Action
2,Day of Defeat,7,76561197970982479,277,2003-05-01,Action
3,Deathmatch Classic,0,76561197970982479,277,2001-06-01,Action
4,Half-Life: Opposing Force,0,76561197970982479,277,1999-11-01,Action


In [7]:
df_merged.to_parquet('../Datasets/userforgenre_endpoint.parquet', compression='snappy', index=False)